In [1]:
!kaggle datasets download -d  harishvutukuri/dogs-vs-wolves -p data

dogs-vs-wolves.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
!unzip -q -u data/dogs-vs-wolves.zip -d data/dogs-vs-wolves

In [3]:
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile # TF temporal bugfix

In [4]:
%load_ext tensorboard

## Imports

In [5]:
import os

# Torch #
import torch

import torchero
from torchero.models.vision import BinaryImageClassificationModel # Training the model
from torchero.callbacks import ProgbarLogger # Model supervising
from torchero.callbacks.tensorboard import TensorBoardLogger # Tensorboard metrics training
from torchero.utils.data import train_test_split
from torchero.utils.vision import show_imagegrid_dataset, transforms, datasets
from torchero.utils.tensorboard import SummaryWriter, write_imagegrid_dataset, write_model
from torchero.hparams import FixedP

from matplotlib import pyplot as plt

In [6]:
!rm -rf runs/

In [7]:
writer = SummaryWriter('runs/Dogs vs Wolves')

## Training dataset

In [8]:
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor()])

In [9]:
dataset = datasets.ImageFolder(root='data/dogs-vs-wolves/data', transform=transform)

In [10]:
train_ds, val_ds = train_test_split(dataset)

In [11]:
write_imagegrid_dataset(writer, 'train_dataset', train_ds)

In [12]:
write_imagegrid_dataset(writer, 'test_dataset', val_ds)

In [13]:
%tensorboard --logdir runs/

## Define model

### VGGs

In [14]:
for num_layers in [11, 13]:
    model = BinaryImageClassificationModel.from_pretrained('vgg' + str(num_layers), num_outputs=1, transform=transform)
    model.labels = ['is_wolf']
    writer = SummaryWriter('runs/Dogs vs Wolves/vgg' + str(num_layers))
    write_model(writer, model.model, train_ds[0][0].unsqueeze(0))
    model.compile(optimizer='adam',
                  hparams={'num_layers': FixedP(num_layers),
                           'model_name': FixedP('vgg')},
                  callbacks=[ProgbarLogger(notebook=True),
                             TensorBoardLogger(writer)]).cuda()
    history = model.fit(train_ds,
          val_ds,
          epochs=3,
          batch_size=32)
    model.to('cpu')
    torch.cuda.empty_cache()
    writer.close()

/usr/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1153.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


  0%|          | 0/3 [00:00<?, ?epoch/s]

  0%|          | 0/50 [00:00<?, ? batchs/s]

  0%|          | 0/50 [00:00<?, ? batchs/s]

  0%|          | 0/50 [00:00<?, ? batchs/s]

  0%|          | 0/3 [00:00<?, ?epoch/s]

  0%|          | 0/50 [00:00<?, ? batchs/s]

  0%|          | 0/50 [00:00<?, ? batchs/s]

  0%|          | 0/50 [00:00<?, ? batchs/s]

### Resnets

In [15]:
for num_layers in [18, 34, 50]:
    model = BinaryImageClassificationModel.from_pretrained('resnet' + str(num_layers), num_outputs=1, transform=transform)
    model.labels = ['is_wolf']
    writer = SummaryWriter('runs/Dogs vs Wolves/resnet' + str(num_layers))
    write_model(writer, model.model, train_ds[0][0].unsqueeze(0))
    model.compile(optimizer='adam',
                  hparams={'num_layers': FixedP(num_layers),
                           'model_name': FixedP('resnet')},
                  callbacks=[ProgbarLogger(notebook=True),
                             TensorBoardLogger(writer)]).cuda()
    history = model.fit(train_ds,
          val_ds,
          epochs=3,
          batch_size=64)
    model.to('cpu')
    torch.cuda.empty_cache()
    writer.close()

  0%|          | 0/3 [00:00<?, ?epoch/s]

  0%|          | 0/25 [00:00<?, ? batchs/s]

  0%|          | 0/25 [00:00<?, ? batchs/s]

  0%|          | 0/25 [00:00<?, ? batchs/s]

  0%|          | 0/3 [00:00<?, ?epoch/s]

...

  0%|          | 0/25 [00:00<?, ? batchs/s]

  0%|          | 0/3 [00:00<?, ?epoch/s]

  0%|          | 0/25 [00:00<?, ? batchs/s]

  0%|          | 0/25 [00:00<?, ? batchs/s]

  0%|          | 0/25 [00:00<?, ? batchs/s]